In [1]:
import pandas as pd
import glob
import os

In [3]:
path_2023 = "rawdata/MBTA_Bus_Arrival_Departure_Times_2023"
path_2024 = "rawdata/MBTA_Bus_Arrival_Departure_Times_2024"

files_2023 = glob.glob(os.path.join(path_2023, "*.csv"))  #match and get all csv files under the variables
files_2024 = glob.glob(os.path.join(path_2024, "*.csv"))

all_files = files_2023 + files_2024 #merge two files

print("number of files :", len(all_files)) # sanity check the number of files, it should print 24 
for f in all_files:
    print(f) #should return all files

dataframes = []
for i, file in enumerate(all_files):
    df = pd.read_csv(file)
    dataframes.append(df)
    print(f"Loaded {i+1}/{len(all_files)}: {file}")   #to observe the process clearly

bus_all = pd.concat(dataframes, ignore_index=True)   # merge the files by rows, ignore_index= True: generate new row numbers

bus_all = bus_all.drop_duplicates() #remove all duplicate row information


number of files : 24
rawdata/MBTA_Bus_Arrival_Departure_Times_2023/MBTA-Bus-Arrival-Departure-Times_2023-01.csv
rawdata/MBTA_Bus_Arrival_Departure_Times_2023/MBTA-Bus-Arrival-Departure-Times_2023-03.csv
rawdata/MBTA_Bus_Arrival_Departure_Times_2023/MBTA-Bus-Arrival-Departure-Times_2023-02.csv
rawdata/MBTA_Bus_Arrival_Departure_Times_2023/MBTA-Bus-Arrival-Departure-Times_2023-06.csv
rawdata/MBTA_Bus_Arrival_Departure_Times_2023/MBTA-Bus-Arrival-Departure-Times_2023-12.csv
rawdata/MBTA_Bus_Arrival_Departure_Times_2023/MBTA-Bus-Arrival-Departure-Times_2023-07.csv
rawdata/MBTA_Bus_Arrival_Departure_Times_2023/MBTA-Bus-Arrival-Departure-Times_2023-11.csv
rawdata/MBTA_Bus_Arrival_Departure_Times_2023/MBTA-Bus-Arrival-Departure-Times_2023-05.csv
rawdata/MBTA_Bus_Arrival_Departure_Times_2023/MBTA-Bus-Arrival-Departure-Times_2023-04.csv
rawdata/MBTA_Bus_Arrival_Departure_Times_2023/MBTA-Bus-Arrival-Departure-Times_2023-10.csv
rawdata/MBTA_Bus_Arrival_Departure_Times_2023/MBTA-Bus-Arrival-Depart

In [4]:
if "service_date" in bus_all.columns:
    bus_all["service_date"] = pd.to_datetime(bus_all["service_date"])  #convert "service_date" to real datetime type.


#generate two new col to save scheduled and actual time and convert the time into real datetime type
bus_all["scheduled_dt_raw"] = pd.to_datetime(bus_all["scheduled"], errors="coerce")
bus_all["actual_dt_raw"]    = pd.to_datetime(bus_all["actual"],    errors="coerce")

#assign hours,minutes, and second values to "schduled_time" and "actual_time"
bus_all["scheduled_time"] = bus_all["scheduled_dt_raw"].dt.time
bus_all["actual_time"]    = bus_all["actual_dt_raw"].dt.time

# create a new col which contains both the year, hour, minutes, and seconds value
bus_all["scheduled_dt"] = bus_all.apply(
    lambda row: pd.Timestamp.combine(row["service_date"].date(), row["scheduled_time"])
    if pd.notnull(row["service_date"]) and pd.notnull(row["scheduled_time"])
    else pd.NaT,
    axis=1
)

bus_all["actual_dt"] = bus_all.apply(
    lambda row: pd.Timestamp.combine(row["service_date"].date(), row["actual_time"])
    if pd.notnull(row["service_date"]) and pd.notnull(row["actual_time"])
    else pd.NaT,
    axis=1
)

bus_all["delay_seconds"] = (
    bus_all["actual_dt"] - bus_all["scheduled_dt"]
).dt.total_seconds()

bus_all["delay_minutes"] = bus_all["delay_seconds"] / 60.0

#create cols about exact hours, weekdays, and if it is weekend, it helps joints with weather.
bus_all["hour"] = bus_all["scheduled_dt"].dt.hour
bus_all["weekday"] = bus_all["scheduled_dt"].dt.weekday  # 0=Mon, 6=Sun
bus_all["is_weekend"] = bus_all["weekday"].isin([5,6]).astype(int)

In [5]:
# keep only reliable and useful features ---
keep_cols = [
    "service_date",
    "hour", "weekday", "is_weekend",
    "route_id", "direction_id",
    "stop_id", "time_point_order", "point_type",
    "scheduled_dt", "actual_dt",
    "delay_seconds", "delay_minutes"
]

bus_clean = bus_all[keep_cols].copy()

# save cleaned dataset 
bus_clean.to_csv("processdata/bus_clean.csv", index=False)
print(" Clean dataset saved as bus_clean.csv")
print("Shape:", bus_clean.shape)

print("\nColumn types:")
print(bus_clean.dtypes) #sanity check

print("\nPreview of cleaned data:")
print(bus_clean.head(10)) # print the first 10 rows

 Clean dataset saved as bus_clean.csv
Shape: (54144994, 13)

Column types:
service_date        datetime64[ns]
hour                         int32
weekday                      int32
is_weekend                   int64
route_id                    object
direction_id                object
stop_id                      int64
time_point_order             int64
point_type                  object
scheduled_dt        datetime64[ns]
actual_dt           datetime64[ns]
delay_seconds              float64
delay_minutes              float64
dtype: object

Preview of cleaned data:
  service_date  hour  weekday  is_weekend route_id direction_id  stop_id  \
0   2023-01-01     6        6           1       01      Inbound      110   
1   2023-01-01     6        6           1       01      Inbound       67   
2   2023-01-01     6        6           1       01      Inbound       72   
3   2023-01-01     6        6           1       01      Inbound       75   
4   2023-01-01     6        6           1       01

In [6]:
bus_clean.head(10)

,service_date,hour,weekday,is_weekend,route_id,direction_id,stop_id,time_point_order,point_type,scheduled_dt,actual_dt,delay_seconds,delay_minutes
0,2023-01-01,6,6,1,01,Inbound,110,1,Startpoint,2023-01-01 06:05:00,2023-01-01 06:05:04,4.0,0.066667
1,2023-01-01,6,6,1,01,Inbound,67,2,Midpoint,2023-01-01 06:09:00,2023-01-01 06:06:28,-152.0,-2.533333
2,2023-01-01,6,6,1,01,Inbound,72,3,Midpoint,2023-01-01 06:12:00,2023-01-01 06:08:57,-183.0,-3.050000
3,2023-01-01,6,6,1,01,Inbound,75,4,Midpoint,2023-01-01 06:15:00,2023-01-01 06:12:41,-139.0,-2.316667
4,2023-01-01,6,6,1,01,Inbound,79,5,Midpoint,2023-01-01 06:19:00,2023-01-01 06:16:35,-145.0,-2.416667
5,2023-01-01,6,6,1,01,Inbound,187,6,Midpoint,2023-01-01 06:21:00,2023-01-01 06:21:39,39.0,0.650000
6,2023-01-01,6,6,1,01,Inbound,59,7,Midpoint,2023-01-01 06:25:00,2023-01-01 06:23:56,-64.0,-1.066667
7,2023-01-01,6,6,1,01,Inbound,110,1,Startpoint,2023-01-01 06:25:00,2023-01-01 06:24:43,-17.0,-0.283333
8,2023-01-01,6,6,1,01,Inbound,67,2,Midpoint,2023-01-01 06:29:00,2023-01-01 06:26:59,-121.0,-2.016667
9,2023-01-01,6,6,1,01,Inbound,62,8,Midpoint,2023-01-01 06:29:00,2023-01-01 06:28:29,-31.0,-0.516667
